### 딥러닝의 이슈
- 데이터가 적고
- 모델이 복잡하면 복잡해질수록 overfitting 문제가 발생한다.
- -> 정규화 방법을 통해 overfitting을 어는정도 잡을수 있다.
- -> dropout 방법 일정 데이터를 랜덤적으로 삭제한다.
- -> 앙상블 기법
    - 베깅은 전체 데이터의 평균을 내서 합친다.
    - 부스팅
- -> early stopping
    - 학습이 어는정도 돼버리면 더 이상 학습을 시키지 않는다.
    
- callback을 통해 epoch이 돌때마다 결과값을 확인.
- 생성된 모델은 hdf5파일로 저장가능하다.
- hdf5파일은 공용파일이다. -> 다양한 언어에서 불러와 모델을 사용할 수 있다.

### https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

In [84]:
# epch를 돌고나서 상태확인
from tensorflow.keras.callbacks import Callback

In [85]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


In [86]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]
# normarllrization -> 정규화
# min-max
"""
X - 0 / 255 - 0

"""


'\nX - 0 / 255 - 0\n\n'

In [87]:
# standardization
# 천장만 학습
train_images = train_images[:1000].reshape(-1, 28 * 28) /255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [76]:
train_images.shape

(1000, 784)

In [88]:
# Define a simple sequential model
# 케라스에서 input, demension을 통해 크기를 맞춰준다.
# Sequential 방법대신 model 을 생성하여 만들수 있다.
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, name="input" ,activation='relu', input_shape=(784,)),
    # overfitting 을 막기위해 사용
    keras.layers.Dropout(0.2),
    # multi - classfication
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                # sparse는 one - hot encdig 대신 쓰인다.
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [89]:
checkpoint_path = "training_1/cp.ckpt1"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 # weight만 저장할것인가
                                                 save_weights_only=True,
                                                 # 주석
                                                 verbose=1)

# keras는 s가 붙으면 리스트를 사용한다.

model.fit(train_images, 
          train_labels,  
          epochs=3,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/3
 864/1000 [========================>.....] - ETA: 0s - loss: 1.2155 - accuracy: 0.6516
Epoch 00001: saving model to training_1/cp.ckpt1
1000/1000 [==============================] - 1s 898us/sample - loss: 1.1316 - accuracy: 0.6770 - val_loss: 0.6975 - val_accuracy: 0.7940
Epoch 2/3
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4305 - accuracy: 0.8761
Epoch 00002: saving model to training_1/cp.ckpt1
1000/1000 [==============================] - 0s 460us/sample - loss: 0.4304 - accuracy: 0.8760 - val_loss: 0.5377 - val_accuracy: 0.8230
Epoch 3/3
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2785 - accuracy: 0.9174
Epoch 00003: saving model to training_1/cp.ckpt1
1000/1000 [==============================] - 0s 408us/sample - loss: 0.2903 - accuracy: 0.9150 - val_loss: 0.5036 - val_accuracy: 0.8430


In [92]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4008 - accuracy: 0.0930
훈련되지 않은 모델의 정확도:  9.30%


In [93]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4257 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


In [94]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 30, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W1008 06:34:56.458989 19284 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt


In [98]:
#노트: 텐서플로는 기본적으로 최근 5개의 체크포인트만 저장합니다.
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0030.ckpt'

In [102]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4809 - accuracy: 0.8760
복원된 모델의 정확도: 87.60%


In [105]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4809 - accuracy: 0.8760
복원된 모델의 정확도: 87.60%


In [106]:
## 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

In [107]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W1008 06:43:09.173171 19284 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W1008 06:43:09.174171 19284 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W1008 06:43:09.176171 19284 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W1008 06:43:09.177171 19284 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W1008 06:43:09.178171 19284 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W1008 06:43:09.180171 19284 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.
W1008 06:43:09.185171 19284 u

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [108]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4809 - accuracy: 0.8760
복원된 모델의 정확도: 87.60%
